In [13]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from scipy.ndimage.morphology import grey_dilation

def read_and_inverse_img(img_name):
    im = Image.open(img_name, 'r').convert('LA')
    pixs = np.array(im)[:, :, 0]
    lorem_inv = np.array(list(map(lambda x: 255 - x, pixs)))
    return lorem_inv

def get_char_map(char, font):
    core = font.getmask(char)
    x, y = core.size
    ch_map = np.reshape(np.array(core), (y, x))
    return ch_map

def write_char_pos(char_ord, char_map, let_pos, lorem_inv, pres):
    e_count = 0
    cor = np.real(
        np.fft.ifft2(
            np.multiply(
                np.fft.fft2(lorem_inv), 
                np.fft.fft2(np.rot90(char_map, 2), np.shape(lorem_inv)))))
    thr = int(np.max(cor)) - pres
    filtered_cor = cor > thr    
    for j in range(0, len(filtered_cor)):
        for k in range(0, len(filtered_cor[j])):
            if filtered_cor[j, k] == True:
                e_count = e_count + 1
                let_pos[j, k] = char_ord
    return e_count

def get_str(let_pos):
    res_str = ""
    space_counter = 0
    for j in range(0, len(let_pos)):
        for k in range(0, len(let_pos[j])):
            if let_pos[j, k] != 0:
                res_str += chr(let_pos[j, k])
            else:
                space_counter += 1
            if space_counter >= 14:
                space_counter = 0
                res_str += ' '
    return res_str

def recognize(img_name):
    lorem_inv = read_and_inverse_img(img_name)
    font = ImageFont.truetype("arial.ttf", 16)
    let_pos = np.zeros(np.shape(lorem_inv), dtype=np.uint8)
    for i in range(ord('a'), ord('z')+1):
        ch_map = get_char_map(chr(i), font)
        write_char_pos(i, ch_map, let_pos, lorem_inv, 50000)
    res_str = get_str(let_pos)
    return res_str

In [2]:
img_name = 'lorem_arial.png'

In [15]:
res_str = recognize(img_name)
print(res_str)